# LargestLLM Router - Inference

This notebook demonstrates the **LargestLLM** baseline router.

## Overview

LargestLLM is a simple baseline router that always routes queries to the largest model in the candidate pool.
This serves as an upper bound for routing performance and a lower bound for cost efficiency.

**Key Characteristics**:
- No training required (deterministic baseline)
- Always selects the largest model by parameter size
- Useful for performance benchmarking
- Highest expected quality, highest cost

## 1. Environment Setup

In [ ]:
# For Google Colab: Clone repository and install dependencies
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/ulab-uiuc/LLMRouter.git
    %cd LLMRouter
    !pip install -e .
    !pip install pyyaml

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

In [ ]:
from llmrouter.models.largest_llm import LargestLLMRouter
from llmrouter.utils import setup_environment
import yaml

setup_environment()
print("Environment setup complete!")

## 2. Configuration

LargestLLM router requires only data paths - no hyperparameters.

| Parameter | Description |
|-----------|-------------|
| `llm_data` | Path to LLM candidate metadata |
| `routing_data_test` | Path to test routing data |

In [ ]:
CONFIG_PATH = "configs/model_config_train/largest_llm.yaml"

with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

print("Current Configuration:")
print("=" * 50)
print(yaml.dump(config, default_flow_style=False))

## 3. Initialize Router

In [ ]:
router = LargestLLMRouter(yaml_path=CONFIG_PATH)

print("Router initialized successfully!")
print(f"Number of LLM candidates: {len(router.llm_data)}")

In [ ]:
# Display available LLM candidates sorted by size
print("Available LLM Candidates (by size):")
print("=" * 60)

llm_list = [(name, info.get('size', 'unknown')) for name, info in router.llm_data.items()]
llm_list_sorted = sorted(llm_list, key=lambda x: float(x[1]) if isinstance(x[1], (int, float)) else 0, reverse=True)

for i, (name, size) in enumerate(llm_list_sorted, 1):
    marker = " <- LARGEST" if i == 1 else ""
    print(f"{i}. {name}: {size}B parameters{marker}")

## 4. Query Routing

LargestLLM always routes to the largest model, regardless of query complexity.

In [ ]:
EXAMPLE_QUERIES = [
    {"query": "What is 2 + 2?"},  # Simple
    {"query": "Explain the theory of general relativity."},  # Medium
    {"query": "Prove P != NP."},  # Complex
]

print("Routing Results:")
print("=" * 60)

for i, query in enumerate(EXAMPLE_QUERIES, 1):
    result = router.route_single(query)
    print(f"{i}. {query['query'][:50]}...")
    print(f"   Routed to: {result['model_name']}")
    print()

## 5. Batch Routing

In [ ]:
# Route test data
test_queries = router.routing_data_test[:10]

print(f"Routing {len(test_queries)} test queries...")
results = router.route(test_queries)

print(f"\nRouting Distribution:")
from collections import Counter
model_counts = Counter([r['model_name'] for r in results])
for model, count in model_counts.most_common():
    print(f"  {model}: {count} ({100*count/len(results):.1f}%)")

## 6. Evaluation

In [ ]:
from llmrouter.evaluator import Evaluator

evaluator = Evaluator(router=router)
metrics = evaluator.eval()

print("\nEvaluation Results:")
print("=" * 50)
for metric_name, value in metrics.items():
    if isinstance(value, float):
        print(f"{metric_name}: {value:.4f}")
    else:
        print(f"{metric_name}: {value}")

## 7. Compare with SmallestLLM

Compare the two baseline routers to understand the performance-cost tradeoff.

In [ ]:
from llmrouter.models.smallest_llm import SmallestLLMRouter

smallest_router = SmallestLLMRouter(yaml_path="configs/model_config_train/smallest_llm.yaml")
smallest_evaluator = Evaluator(router=smallest_router)
smallest_metrics = smallest_evaluator.eval()

print("Comparison: SmallestLLM vs LargestLLM")
print("=" * 60)
print(f"{'Metric':<20} {'SmallestLLM':<15} {'LargestLLM':<15}")
print("-" * 60)

for key in metrics.keys():
    if key in smallest_metrics:
        s_val = smallest_metrics[key]
        l_val = metrics[key]
        if isinstance(s_val, float):
            print(f"{key:<20} {s_val:<15.4f} {l_val:<15.4f}")
        else:
            print(f"{key:<20} {str(s_val):<15} {str(l_val):<15}")

## 8. File-Based Inference

Load queries from a custom file and save results.

In [ ]:
import json

# Load queries from a JSONL file
def load_queries_from_file(file_path):
    """Load queries from a JSONL file."""
    queries = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                queries.append(json.loads(line))
    return queries

# Save results to a JSONL file
def save_results_to_file(results, output_path):
    """Save routing results to a JSONL file."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        for result in results:
            f.write(json.dumps(result, ensure_ascii=False) + '\n')
    print(f"Results saved to: {output_path}")

# Example: Load from your own query file
QUERY_FILE = "data/example_data/query_data/default_query_test.jsonl"
OUTPUT_FILE = "outputs/largest_llm_results.jsonl"

if os.path.exists(QUERY_FILE):
    # Load queries from file
    file_queries = load_queries_from_file(QUERY_FILE)
    print(f"Loaded {len(file_queries)} queries from: {QUERY_FILE}")
    
    # Route queries using route_batch
    file_results = router.route_batch(batch=file_queries[:10])
    print(f"Routed {len(file_results)} queries")
    
    # Save results to file
    save_results_to_file(file_results, OUTPUT_FILE)
    
    # Show sample results
    print(f"\nSample results:")
    for i, result in enumerate(file_results[:3], 1):
        print(f"  {i}. {result.get('query', '')[:40]}... -> {result['model_name']}")
else:
    print(f"Query file not found: {QUERY_FILE}")
    print("Create a JSONL file with format: {\"query\": \"Your question\"}")

## Summary

**LargestLLM Router**:
- Always routes to the largest model
- No training required (deterministic baseline)
- Provides upper bound for performance, lower bound for cost efficiency
- Useful for comparing against learned routing methods

**Use Cases**:
- Baseline comparison for routing research
- Quality-critical applications regardless of cost
- Establishing performance ceiling for routing methods